In [2]:
import json
import hashlib
from datetime import datetime
from breeze_connect import BreezeConnect
import urllib
import pandas as pd
import numpy as np
import math



In [3]:
class icici:
    defalut_key = '73=63fT44781tY4C42L0i3491201v23A'
    default_secret = '11331XF5fm6c5&Y3K2Q99lp28347480+'

    def __init__(self,api_session,key=defalut_key,secret = default_secret):
        self.Key = key
        self.Secret = secret
        self.api_session = api_session

    
    def login(self):
        self.breeze = BreezeConnect(api_key=self.Key)
        self.breeze.generate_session(api_secret=self.Secret,session_token=self.api_session)
        # Generate ISO8601 Date/DateTime String
        import datetime
        iso_date_string = datetime.datetime.strptime("07/12/2023","%d/%m/%Y").isoformat()[:10] + 'T05:30:00.000Z'
        iso_date_time_string = datetime.datetime.strptime("07/12/2023 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + '.000Z'


In [4]:
Key = '73=63fT44781tY4C42L0i3491201v23A'
Secret = '11331XF5fm6c5&Y3K2Q99lp28347480+'
api_session = '37289997'

In [6]:


# Initialize SDK
breeze = BreezeConnect(api_key=Key)

# Obtain your session key from https://api.icicidirect.com/apiuser/login?api_key=YOUR_API_KEY
# Incase your api-key has special characters(like +,=,!) then encode the api key before using in the url as shown below.

#print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus("your_api_key"))

# Generate Session
breeze.generate_session(api_secret=Secret,
                        session_token=api_session)

# Generate ISO8601 Date/DateTime String
import datetime
iso_date_string = datetime.datetime.strptime("07/12/2023","%d/%m/%Y").isoformat()[:10] + 'T05:30:00.000Z'
iso_date_time_string = datetime.datetime.strptime("07/12/2023 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + '.000Z'

In [ ]:
#get customer details
breeze.get_customer_details(api_session=api_session) 

In [ ]:
#get holdings
breeze.get_demat_holdings()

In [ ]:
#get_stock details 
def get_histroy_data_stock(stock_code,from_date,to_date,interval='1day',exchange='NSE'):
    data = breeze.get_historical_data(interval=interval,
                                from_date= f"{from_date}T07:00:00.000Z",
                                to_date= f"{to_date}T07:00:00.000Z",
                                stock_code=stock_code,
                                exchange_code=exchange,
                                product_type="cash")
    if data['Status']==200:
        df = pd.DataFrame(data['Success'])[['datetime','stock_code','open','close','high','low']]
        for x in df.columns:
            try:
                df[x]=df[x].astype(float)
                print(x,'complete')
            except Exception as e:
                print('couldnt perform on',x)
                print('Error',e)
        return df
    else:
        print('got error')
        return data
    
def get_log_return(df):
    df['Log Returns'] = np.log(df['close'] / df['close'].shift(1))
    return df

In [16]:
data = breeze.get_historical_data(interval='1minute',
                                from_date= f"{'2024-03-19'}T07:00:00.000Z",
                                to_date= f"{'2024-03-20'}T07:00:00.000Z",
                                stock_code='ITC',
                                exchange_code='NSE',
                                product_type="cash")

In [18]:
df = pd.DataFrame(data['Success'])
df

,datetime,stock_code,exchange_code,product_type,expiry_date,right,strike_price,open,high,low,close,volume,open_interest,count
0,2024-03-19 09:08:00,ITC,NSE,None,None,None,None,418,418,418,418,2,None,0
1,2024-03-19 09:15:00,ITC,NSE,None,None,None,None,418.15,418.4,418,418.4,521,None,1
2,2024-03-19 09:16:00,ITC,NSE,None,None,None,None,418.4,418.95,418,418.15,2045,None,2
3,2024-03-19 09:17:00,ITC,NSE,None,None,None,None,418.35,418.75,418.1,418.65,4180,None,3
4,2024-03-19 09:18:00,ITC,NSE,None,None,None,None,418.65,419.3,418.45,419,12886,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,2024-03-19 15:56:00,ITC,NSE,None,None,None,None,409.55,409.55,409.55,409.55,314,None,393
394,2024-03-19 15:57:00,ITC,NSE,None,None,None,None,409.55,409.55,409.55,409.55,518,None,394
395,2024-03-19 15:58:00,ITC,NSE,None,None,None,None,409.55,409.55,409.55,409.55,24,None,395
396,2024-03-19 15:59:00,ITC,NSE,None,None,None,None,409.55,409.55,409.55,409.55,106,None,396


In [ ]:
data_1year = get_histroy_data_stock(_stock_code,'2023-02-12','2024-03-15')
data_30days = get_histroy_data_stock(_stock_code,'2024-02-12','2024-03-15')
data_1year

In [ ]:

data_1year = get_log_return(data_1year)
data_30days = get_log_return(data_30days)
data_1year


In [ ]:
print(30,data_30days['Log Returns'].mean())
print('1 year',data_1year['Log Returns'].mean())

In [ ]:
def range_for_days(df,days):
    original = df
    last_price = original['close'].iloc[-1]
    df = df['Log Returns']
    mean = df.mean()
    sd = df.std()
    print('Calculated mean',mean,'\n')
    print('calculated sd', sd,'\n')
    average = days*mean
    std = days**(0.5)*sd
    print('adjusted average for',days,'is',average*100,'%\n')
    print('adjusted std for',days,'is',std*100,'%\n')

    ranges = {}
    for x,y in zip(range(1,4),[68,95,99.7]):
        lower_range = average-std*x
        upper_range = average+std*x
        lower_range_price = last_price*math.exp(lower_range)
        upper_range_price = last_price*math.exp(upper_range)

        
        print(f'Range for {x}sd {y}% confidence level is {lower_range} -- {upper_range}')
        print(f'Range in price for {x}sd {y}% confidence level is {lower_range_price} -- {upper_range_price}\n')
        ranges[y] = [lower_range_price,upper_range_price]
    
    return ranges

range_for_days(data_1year,30)


In [ ]:
range_for_days(data_1year,6)


In [ ]:
range_for_days(data_30days,6)

In [ ]:
8337*math.exp(-0.0695)

In [ ]:
#get bank details
breeze.get_funds()